# 금통위 의사록 어조 분석

1. 어조록에서 금리상승 및 하락 문장 추출
    -up_count : 금리상승 단어 등장 횟수 / down_count : 금리하락 단어 등장 횟수
    -pos_count : 금리상승 문장 등장 횟수 / neg_count : 금리하락 문장 등장 횟수

In [ ]:
import pandas as pd
df_minites = pd.read_csv(r"C:\Users\HanBi\Desktop\2021\project1\crawling\minutes_data_save.csv")
df_minites=df_minites.drop("Unnamed: 0",axis=1)

from ekonlpy.sentiment import MPCK
mpck = MPCK()

def make_ngram(sen):
    tokens=mpck.tokenize(sen)
    ngrams=mpck.ngramize(tokens)
    total_list=tokens+ngrams
    return total_list

df_Pos=pd.read_csv(r"C:\Users\HanBi\Desktop\2021\project1\crawling\긍정사전.csv")
df_Neg=pd.read_csv(r"C:\Users\HanBi\Desktop\2021\project1\crawling\부정사전.csv")

Pos_list=df_Pos['ngram'].to_list()
Neg_list=df_Neg['ngram'].to_list()

#df_minites=df_minites.drop(['up_count','down_count'],axis=1)

df_minites['pos_count']=np.nan
df_minites['neg_count']=np.nan

for i in range(len(df_minites)):
    #의사록 어조
    pos_count=0
    neg_count=0
    sentences=df_minites['content'][i].split("@@@")
    for sentence in sentences:
        #문장 어조 
        up_count=0
        down_count=0
        ngram_list=make_ngram(sentence)
        for item in ngram_list:
            if item in Pos_list:
                up_count=up_count+1
            elif item in Neg_list:
                down_count=down_count+1
        try:
            #print(up_count, down_count)
            sentence_tone = (up_count-down_count)/(up_count+down_count)
        except Exception as e:
            #print(e)
            sentence_tone=0
        if sentence_tone >0:
            pos_count=pos_count+1
        elif sentence_tone<0:
            neg_count=neg_count+1
    df_minites.loc[i, ['pos_count']]=pos_count
    df_minites.loc[i, ['neg_count']]=neg_count
    print(i)

df_minites.to_csv(r"C:\Users\HanBi\Desktop\2021\project1\crawling\의사록1.csv")

2. 의사록별 어조 및 라벨링하기

In [ ]:
import pandas as pd

df=pd.read_csv(r"C:\Users\HanBi\Desktop\2021\project1\crawling\의사록1.csv")

df['TONE_doc']=(df['pos_count']-df['neg_count'])/(df['pos_count']+df['neg_count'])

def get_label(df):
    if df['TONE_doc'] > 0:
        return "hawk"
    elif df['TONE_doc'] < 0:
        return "dove"
    
df['TONE_Doc_Label']=df.apply(get_label,axis=1)


df.to_csv(r"C:\Users\HanBi\Desktop\2021\project1\crawling\최종의사록어조.csv", encoding='utf-8', index=False)


3. 상관계수 구하기(기준금리-의사록tone)

In [ ]:
dfdf = pd.read_csv(r'C:\Users\kyt50\anaconda3\kdigital\project\최종의사록어조.csv', encoding='utf-8')
dfdf1 = pd.read_csv(r'C:\Users\kyt50\anaconda3\kdigital\project\기준금리.csv', encoding='utf-8')
subset = dfdf[['date', 'TONE_doc']]               #dfdf데이터프레임에서 날짜와 톤만 추출하는 데이터프레임 만들기
cor_df = pd.merge(dfdf1, subset, how='inner', on='date') # date를 기준으로 inner조인하여 머지하기
import scipy.stats as stats                       #상관계수 구하기
corr = stats.pearsonr(cor_df.TONE_doc ,cor_df.standardrate)
corr